In [1]:
import pandas as pd
import numpy as np

file_path = r'C:\Users\xycy1\Desktop\PeriodRes_AAAI\code\Comparison\total_acc.xlsx'
df_acc = pd.read_excel(file_path, sheet_name=0, engine='openpyxl')

In [2]:
len(df_acc), df_acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     128 non-null    object 
 1   TimesNet       128 non-null    float64
 2   Inceptiontime  128 non-null    float64
 3   LSTNet         128 non-null    float64
 4   FEDformer      128 non-null    float64
 5   Minirocket     128 non-null    float64
 6   Rocket         128 non-null    float64
 7   Hivecote       128 non-null    float64
 8   Hydra          128 non-null    float64
 9   rmESN          128 non-null    float64
 10  PARC           128 non-null    float64
dtypes: float64(10), object(1)
memory usage: 11.1+ KB


(128, None)

In [3]:
# 将第一列设置为索引
df_acc.set_index(df_acc.columns[0], inplace=True)

# 需要删除的行的列表
indices_to_remove = ['SyntheticControl', 'TwoPatterns', 'SmoothSubspace', 
                     'GestureMidAirD1', 'GestureMidAirD3', 'UWaveGestureLibraryX', 
                     'UWaveGestureLibraryZ']

# 删除指定索引的行
df_acc.drop(indices_to_remove, errors='ignore', inplace=True)

In [4]:
len(df_acc), df_acc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121 entries, ACSF1 to Yoga
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TimesNet       121 non-null    float64
 1   Inceptiontime  121 non-null    float64
 2   LSTNet         121 non-null    float64
 3   FEDformer      121 non-null    float64
 4   Minirocket     121 non-null    float64
 5   Rocket         121 non-null    float64
 6   Hivecote       121 non-null    float64
 7   Hydra          121 non-null    float64
 8   rmESN          121 non-null    float64
 9   PARC           121 non-null    float64
dtypes: float64(10)
memory usage: 10.4+ KB


(121, None)

In [5]:
def print_total_accuracy(df):
    for dataset_name in df.index:
        accuracies = df.loc[dataset_name, :]
        max_acc = accuracies.max()
        formatted_accuracies = [f"\\textbf{{{acc:.4f}}}" if acc == max_acc else "%.4f" % acc for acc in accuracies]
        row_str = f"{dataset_name} & {' & '.join(formatted_accuracies)} \\\\"
        print(row_str)

print_total_accuracy(df_acc)

ACSF1 & 0.3500 & 0.8800 & 0.8500 & 0.5600 & 0.9000 & 0.8800 & \textbf{0.9400} & 0.8800 & 0.8000 & 0.8500 \\
Adiac & 0.4859 & 0.7494 & 0.7442 & 0.2890 & 0.8184 & 0.7852 & 0.8107 & \textbf{0.8235} & 0.4655 & 0.7545 \\
AllGestureWiimoteX & 0.4629 & 0.7671 & 0.6114 & 0.5129 & 0.7100 & \textbf{0.7743} & 0.6757 & 0.7014 & 0.4686 & 0.6071 \\
AllGestureWiimoteY & 0.5229 & \textbf{0.7871} & 0.6657 & 0.5643 & 0.7100 & 0.7643 & 0.6843 & 0.7600 & 0.5086 & 0.7214 \\
AllGestureWiimoteZ & 0.3529 & \textbf{0.7843} & 0.5657 & 0.4243 & 0.6871 & 0.7586 & 0.6486 & 0.7143 & 0.5386 & 0.6929 \\
ArrowHead & 0.7829 & \textbf{0.8629} & 0.8057 & 0.7143 & 0.8229 & 0.8229 & 0.8457 & 0.8114 & 0.7371 & 0.8571 \\
BME & 0.9933 & 0.9933 & 0.9333 & 0.8133 & \textbf{1.0000} & \textbf{1.0000} & 0.9000 & \textbf{1.0000} & 0.7400 & \textbf{1.0000} \\
Beef & 0.8667 & 0.7000 & 0.7000 & 0.8000 & 0.8667 & 0.8000 & 0.9000 & 0.8333 & 0.7000 & \textbf{0.9333} \\
BeetleFly & 0.9000 & 0.8000 & 0.9500 & 0.8500 & 0.9500 & 0.9000 & 0.9

In [6]:
from scipy.stats import rankdata

def rank_counts(df, k):
    all_ranks = []
    for dataset_name in df.index:
        accuracies = df.loc[dataset_name, :] # 取某一行的全部值 type:series
        ranks = rankdata(-accuracies, method='min') # 对数据进行排名，将较小（-较大）的排在前头
        all_ranks.append(ranks)

    all_ranks_df = pd.DataFrame(all_ranks)
    counts = (all_ranks_df <= k).sum(axis=0).tolist()  # 计算排名第一的次数
    count_str = f"Num. of Top-{k} & {' & '.join(map(str, counts))} \\\\"
    print(f"Column Names & {' & '.join(list(df.columns))}")
    print(count_str)

rank_counts(df_acc, 1)

Column Names & TimesNet & Inceptiontime & LSTNet & FEDformer & Minirocket & Rocket & Hivecote & Hydra & rmESN & PARC
Num. of Top-1 & 8 & 19 & 11 & 0 & 16 & 22 & 28 & 25 & 5 & 42 \\


In [9]:
def one_to_one_comparison(df):
    ours_index = df.shape[1]
    wins = [0] * ours_index
    ties = [0] * ours_index
    losses = [0] * ours_index

    for i in range(0, len(df)):
        ours_accuracy = df.iloc[i, ours_index - 1]
        for j in range(ours_index - 1):
            if df.iloc[i, j] > ours_accuracy:
                losses[j] += 1
            elif df.iloc[i, j] < ours_accuracy:
                wins[j] += 1
            else:
                ties[j] += 1

    wins[ours_index - 1] = "-"
    ties[ours_index - 1] = "-"
    losses[ours_index - 1] = "-"

    wins_str = f"Ours 1-to-1 Wins & {' & '.join(map(str, wins))} \\\\"
    ties_str = f"Ours 1-to-1 Ties & {' & '.join(map(str, ties))} \\\\"
    losses_str = f"Ours 1-to-1 Losses & {' & '.join(map(str, losses))} \\\\"
    print(f"Column Names & {' & '.join(list(df.columns))}")
    print(wins_str)
    print(ties_str)
    print(losses_str)


one_to_one_comparison(df_acc)

Column Names & TimesNet & Inceptiontime & LSTNet & FEDformer & Minirocket & Rocket & Hivecote & Hydra & rmESN & PARC
Ours 1-to-1 Wins & 106 & 69 & 95 & 112 & 65 & 59 & 62 & 63 & 111 & - \\
Ours 1-to-1 Ties & 6 & 6 & 4 & 2 & 14 & 15 & 6 & 10 & 6 & - \\
Ours 1-to-1 Losses & 9 & 46 & 22 & 7 & 42 & 47 & 53 & 48 & 4 & - \\


In [10]:
def average_rank(df):
    all_ranks = []
    for data_name in df.index:
        accuracies = df.loc[data_name, :]
        # ranks = rankdata(-accuracies, method='min')
        ranks = rankdata(-accuracies, method='average')
        all_ranks.append(ranks)

    avg_ranks = pd.DataFrame(all_ranks).mean().tolist()
    max_avg_rank = min(avg_ranks)  # 由于排名越小越好，取最小值
    formatted_avg_ranks = [f"\\textbf{{{rank:.3f}}}" if rank == max_avg_rank else f"{rank:.3f}" for rank in avg_ranks]
    avg_rank_str = f"Avg. Rank & {' & '.join(formatted_avg_ranks)} \\\\"
    print(f"Column Names & {' & '.join(list(df.columns))}")
    print(avg_rank_str)

def average_accuracy(df):
    avg_accuracies = df.mean()
    max_avg_acc = avg_accuracies.max()
    formatted_avg_accuracies = [f"\\textbf{{{acc:.3f}}}" if acc == max_avg_acc else f"{acc:.3f}" for acc in
                                avg_accuracies]
    avg_str = f"Avg. Acc. & {' & '.join(formatted_avg_accuracies)} \\\\"
    print(avg_str)


average_rank(df_acc)
average_accuracy(df_acc)

Column Names & TimesNet & Inceptiontime & LSTNet & FEDformer & Minirocket & Rocket & Hivecote & Hydra & rmESN & PARC
Avg. Rank & 7.769 & 4.752 & 6.773 & 7.822 & 4.285 & 3.884 & 4.136 & 4.269 & 7.727 & \textbf{3.583} \\
Avg. Acc. & 0.707 & 0.837 & 0.758 & 0.716 & 0.848 & 0.851 & 0.844 & 0.848 & 0.737 & \textbf{0.856} \\


In [11]:
from scipy.stats import wilcoxon

def print_p_values(df):
    algorithms = df.columns
    accuracies = df
    num_algorithms = accuracies.shape[1]
    p_values_matrix = np.zeros((num_algorithms, num_algorithms))

    for i in range(num_algorithms):
        for j in range(num_algorithms):
            if i != j:
                stat, p_value = wilcoxon(accuracies.iloc[:, i], accuracies.iloc[:, j])
                p_values_matrix[i, j] = p_value
            else:
                p_values_matrix[i, j] = np.nan  # 自己与自己比较，用NaN表示

    latex_str = ""

    for i in range(len(algorithms)):
        row = [algorithms[i]]
        for j in range(len(algorithms)):
            if i == j:
                row.append("-")
            else:
                p_value_str = f"{p_values_matrix[i, j]:.2E}"
                if p_values_matrix[i, j] > 0.05:
                    p_value_str = f"\\cellcolor{{gray!25}}{p_value_str}"
                row.append(p_value_str)
        latex_str += " & ".join(row) + " \\\\\n"

    print(latex_str)

print_p_values(df_acc)

TimesNet & - & 8.40E-14 & 2.54E-05 & \cellcolor{gray!25}1.89E-01 & 1.54E-16 & 7.29E-17 & 2.29E-15 & 2.17E-16 & \cellcolor{gray!25}1.73E-01 & 3.66E-18 \\
Inceptiontime & 8.40E-14 & - & 1.81E-10 & 9.63E-15 & \cellcolor{gray!25}5.26E-02 & 2.00E-03 & \cellcolor{gray!25}8.77E-02 & \cellcolor{gray!25}1.57E-01 & 5.11E-13 & 1.07E-02 \\
LSTNet & 2.54E-05 & 1.81E-10 & - & 5.58E-05 & 3.71E-12 & 8.72E-13 & 7.25E-11 & 1.05E-10 & 5.15E-03 & 3.21E-14 \\
FEDformer & \cellcolor{gray!25}1.89E-01 & 9.63E-15 & 5.58E-05 & - & 1.42E-17 & 1.52E-18 & 5.96E-17 & 9.28E-17 & \cellcolor{gray!25}4.65E-01 & 1.14E-19 \\
Minirocket & 1.54E-16 & \cellcolor{gray!25}5.26E-02 & 3.71E-12 & 1.42E-17 & - & \cellcolor{gray!25}1.31E-01 & \cellcolor{gray!25}7.51E-01 & \cellcolor{gray!25}4.63E-01 & 2.83E-15 & 3.93E-02 \\
Rocket & 7.29E-17 & 2.00E-03 & 8.72E-13 & 1.52E-18 & \cellcolor{gray!25}1.31E-01 & - & \cellcolor{gray!25}7.95E-01 & \cellcolor{gray!25}1.06E-01 & 3.03E-16 & \cellcolor{gray!25}2.71E-01 \\
Hivecote & 2.29E-15 &

# f1score

In [10]:
file_path = r'C:\Users\xycy1\Desktop\PeriodRes_AAAI\code\Comparison\total_f1score.xlsx'
df_f1score = pd.read_excel(file_path, sheet_name=0, engine='openpyxl')
df_f1score.set_index(df_f1score.columns[0], inplace=True)

In [11]:
def match_indices(df_1, df_2):
    # 获取df_acc的索引
    target_indices = df_1.index
    
    # 过滤df_time，使其只保留在df_acc中存在的索引
    df_2_new = df_2[df_2.index.isin(target_indices)]
    
    return df_2_new

df_f1score = match_indices(df_acc, df_f1score)

In [13]:
file_path = r'C:\Users\xycy1\Desktop\PeriodRes_AAAI\code\Comparison\total_time.xlsx'
df_times = pd.read_excel(file_path, sheet_name=0, engine='openpyxl')
df_times.set_index(df_times.columns[0], inplace=True)

def match_indices(df_1, df_2):
    # 获取df_acc的索引
    target_indices = df_1.index
    
    # 过滤df_time，使其只保留在df_acc中存在的索引
    df_2_new = df_2[df_2.index.isin(target_indices)]
    
    return df_2_new

df_times = match_indices(df_acc, df_times)


In [19]:
def average_epoch_time(df):
    avg_accuracies = df.mean()
    max_avg_acc = avg_accuracies.min()
    formatted_avg_accuracies = [f"\\textbf{{{acc:.2f}}}" if acc == max_avg_acc else f"{acc:.2f}" for acc in
                                avg_accuracies]
    avg_str = f"Avg. Time(s) & {' & '.join(formatted_avg_accuracies)} \\\\"
    print(avg_str)

In [22]:
average_rank(df_acc)
average_accuracy(df_acc)
average_accuracy(df_f1score)
average_epoch_time(df_times)
one_to_one_comparison(df_acc)

Column Names & TimesNet & Inceptiontime & LSTNet & FEDformer & Minirocket & Rocket & Hivecote & Hydra & rmESN & PARC
Avg. Rank & 7.769 & 4.752 & 6.773 & 7.822 & 4.285 & 3.884 & 4.136 & 4.269 & 7.727 & \textbf{3.583} \\
Avg. Acc. & 0.707 & 0.837 & 0.758 & 0.716 & 0.848 & 0.851 & 0.844 & 0.848 & 0.737 & \textbf{0.856} \\
Avg. Acc. & 0.689 & 0.832 & 0.753 & 0.701 & 0.844 & 0.847 & 0.837 & 0.844 & 0.726 & \textbf{0.852} \\
Avg. Time(s) & 367.03 & 393.62 & 20.51 & 106.63 & 4.35 & 30.50 & 9688.62 & 2.80 & \textbf{0.25} & 95.55 \\
Column Names & TimesNet & Inceptiontime & LSTNet & FEDformer & Minirocket & Rocket & Hivecote & Hydra & rmESN & PARC
Ours 1-to-1 Wins & 69 & 95 & 112 & 65 & 59 & 62 & 63 & 111 & - \\
Ours 1-to-1 Ties & 6 & 4 & 2 & 14 & 15 & 6 & 10 & 6 & - \\
Ours 1-to-1 Losses & 46 & 22 & 7 & 42 & 47 & 53 & 48 & 4 & - \\


## Tanspose

In [42]:
def collect_info(df):
    df_index = df.columns
    avg_acc = df.mean(axis=0)
    max_avg_acc = avg_acc.max()


    all_ranks = []
    for data_name in df.index:
        accuracies = df.loc[data_name, :]
        ranks = rankdata(-accuracies, method='average')
        all_ranks.append(ranks)
    avg_ranks = pd.DataFrame(all_ranks).mean().tolist()
    # max_avg_rank = min(avg_ranks)  # 由于排名越小越好，取最小值


    num_algorithms = df.shape[1]
    p_values_matrix = np.zeros((num_algorithms, num_algorithms))
    for i in range(num_algorithms):
        for j in range(num_algorithms):
            if i != j:
                stat, p_value = wilcoxon(df.iloc[:, i], df.iloc[:, j])
                p_values_matrix[i, j] = p_value
            else:
                p_values_matrix[i, j] = np.nan  # 自己与自己比较，用NaN表示
    p_value_ours = p_values_matrix[-1, :].tolist()


    ours_index = df.shape[1]
    wins = [0] * ours_index
    ties = [0] * ours_index
    losses = [0] * ours_index
    for i in range(0, len(df)):
        ours_accuracy = df.iloc[i, ours_index - 1]
        for j in range(ours_index - 1):
            if df.iloc[i, j] > ours_accuracy:
                losses[j] += 1
            elif df.iloc[i, j] < ours_accuracy:
                wins[j] += 1
            else:
                ties[j] += 1
    wins[ours_index - 1] = "-"
    ties[ours_index - 1] = "-"
    losses[ours_index - 1] = "-"

    conbine = []
    for a, b, c in zip(wins, ties, losses):
        conbine.append(f'{a}/{b}/{c}')
    # 创建一个新的 DataFrame
    new_df = pd.DataFrame(avg_acc, index=df_index, columns=['Average'])
    # 添加 avg_ranks 到 new_df
    new_df['Avg Rank'] = avg_ranks
    new_df['p-Value'] = p_value_ours
    new_df['wins/ties/losses'] = conbine
    return new_df


acc_info = collect_info(df_acc)
print(acc_info)

                Average  Avg Rank       p-Value wins/ties/losses
TimesNet       0.706629  7.768595  3.655248e-18          106/6/9
Inceptiontime  0.836697  4.752066  1.074891e-02          69/6/46
LSTNet         0.757660  6.772727  3.208298e-14          95/4/22
FEDformer      0.715691  7.822314  1.141994e-19          112/2/7
Minirocket     0.847786  4.285124  3.933975e-02         65/14/42
Rocket         0.851035  3.884298  2.706396e-01         59/15/47
Hivecote       0.843570  4.136364  2.684871e-01          62/6/53
Hydra          0.847957  4.268595  9.294927e-02         63/10/48
rmESN          0.737442  7.727273  5.292879e-20          111/6/4
PARC           0.856104  3.582645           NaN            -/-/-


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
from scipy.stats import rankdata

# 读取xlsx到df
def read_file(file_path, drop_datasets):
    df = pd.read_excel(file_path, sheet_name=0, engine='openpyxl')
    # 将第一列设置为索引
    df.set_index(df.columns[0], inplace=True)
    # 删除指定索引的行
    df.drop(drop_datasets, errors='ignore', inplace=True)
    return df



def collect_info(df):
    df_index = df.columns
    avg_acc = df.mean(axis=0)
    max_avg_acc = avg_acc.max()


    all_ranks = []
    for data_name in df.index:
        accuracies = df.loc[data_name, :]
        ranks = rankdata(-accuracies, method='average')
        all_ranks.append(ranks)
    avg_ranks = pd.DataFrame(all_ranks).mean().tolist()
    # max_avg_rank = min(avg_ranks)  # 由于排名越小越好，取最小值


    num_algorithms = df.shape[1]
    p_values_matrix = np.zeros((num_algorithms, num_algorithms))
    for i in range(num_algorithms):
        for j in range(num_algorithms):
            if i != j:
                stat, p_value = wilcoxon(df.iloc[:, i], df.iloc[:, j])
                p_values_matrix[i, j] = p_value
            else:
                p_values_matrix[i, j] = np.nan  # 自己与自己比较，用NaN表示
    p_value_ours = p_values_matrix[-1, :].tolist()


    ours_index = df.shape[1]
    wins = [0] * ours_index
    ties = [0] * ours_index
    losses = [0] * ours_index
    for i in range(0, len(df)):
        ours_accuracy = df.iloc[i, ours_index - 1]
        for j in range(ours_index - 1):
            if df.iloc[i, j] > ours_accuracy:
                losses[j] += 1
            elif df.iloc[i, j] < ours_accuracy:
                wins[j] += 1
            else:
                ties[j] += 1
    wins[ours_index - 1] = "-"
    ties[ours_index - 1] = "-"
    losses[ours_index - 1] = "-"


    # 创建一个新的 DataFrame
    new_df = pd.DataFrame(avg_acc, index=df_index, columns=['Average'])
    # 添加 avg_ranks 到 new_df
    new_df['Avg Rank'] = avg_ranks
    new_df['p-Value'] = p_value_ours
    new_df['wins|ties|losses'] = wins
    new_df['ties'] = ties
    new_df['losses'] = losses

    return new_df

In [30]:
acc_file_path = r'C:\Users\xycy1\Desktop\PeriodRes_AAAI\code\Comparison\total_acc.xlsx'
f1_file_path = r'C:\Users\xycy1\Desktop\PeriodRes_AAAI\code\Comparison\total_f1score.xlsx'
indices_to_remove = ['SyntheticControl', 'TwoPatterns', 'SmoothSubspace', 
                    'GestureMidAirD1', 'GestureMidAirD3', 'UWaveGestureLibraryX', 
                    'UWaveGestureLibraryZ']
df_acc = read_file(acc_file_path, indices_to_remove)
df_f1 = read_file(f1_file_path, indices_to_remove)
acc_info = collect_info(df_acc)
f1_info = collect_info(df_f1)


In [33]:
total_info = pd.concat([acc_info, f1_info], axis=1)
print(total_info)

                Average  Avg Rank       p-Value wins ties losses   Average  \
TimesNet       0.706629  7.768595  3.655248e-18  106    6      9  0.689032   
Inceptiontime  0.836697  4.752066  1.074891e-02   69    6     46  0.832082   
LSTNet         0.757660  6.772727  3.208298e-14   95    4     22  0.752550   
FEDformer      0.715691  7.822314  1.141994e-19  112    2      7  0.701460   
Minirocket     0.847786  4.285124  3.933975e-02   65   14     42  0.843707   
Rocket         0.851035  3.884298  2.706396e-01   59   15     47  0.847136   
Hivecote       0.843570  4.136364  2.684871e-01   62    6     53  0.836788   
Hydra          0.847957  4.268595  9.294927e-02   63   10     48  0.843532   
rmESN          0.737442  7.727273  5.292879e-20  111    6      4  0.725680   
PARC           0.856104  3.582645           NaN    -    -      -  0.851549   

               Avg Rank       p-Value wins ties losses  
TimesNet       7.814050  1.484744e-18  109    3      9  
Inceptiontime  4.702479  1.